In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# %matplotlib inline
from astropy.io import fits
import datetime
from astropy import units as u
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
import time

In [2]:
from some_func import cart2spheric, spheric2cart, spheric_coordinates_rotate, hor2pix_keo, bright_relay_keo

In [3]:
img_num=3215
# img_num=3351

In [4]:
pic_save_name='/home/ashindin/temp/' + str(img_num) + '.png'

In [5]:
fit_filename='/mnt/Work_disk/Owncloud/0000_Two_points/DATA/02_KEO/140826/20140826_' + str(img_num) + '.fit'
dark_filename='/mnt/Work_disk/Owncloud/0000_Two_points/DATA/02_KEO/140826/dark/keo_aug26_dark_median.fit'
BS6300_catalog_filename='Haritonov_6300BS_catalog.npz'

In [6]:
# Координаты расположения KEO
lat_keo=55.9305361 # ШИРОТА
lon_keo=48.7444861 # ДОЛГОТА
h_keo=91.0
KEO_site=EarthLocation(lat=lat_keo*u.deg, lon=lon_keo*u.deg, height=h_keo*u.m)

# Ориентация KEO - 26 августа
az0_keo = 53.3287070263663878*np.pi/180;
alt0_keo = 88.4757875994250895*np.pi/180;
a_keo = (   1.7314835575767389,   -0.0028386665243216,   -0.0039615529590125)
b_keo = (   -0.2861392093442666,    0.0039634986844613,   -0.0028425834931615)
c_keo = ( 254.7431859558771237, -119.5787646993292981,  166.6502675414942019)
d_keo = ( 254.5346368979857630, -166.7359040180428167, -119.4151027599713473)

In [7]:
npzfile=np.load(BS6300_catalog_filename)
CAT_FIELDS=npzfile['CAT_FIELDS']
CAT_6300BS=npzfile['CAT_6300BS']
CAT_SPTYPE=npzfile['CAT_SPTYPE']
print(CAT_FIELDS)

['HARITONOV_ID' 'BS_ID' 'RA (icrs, deg)' 'DEC (icrs, deg)' 'MAG_V'
 '6300 Flux (erg/(sm^2 * s *  sm))']


In [8]:
fid_fit=fits.open(fit_filename);
date_obs_str=fid_fit[0].header["DATE-OBS"]
img=fid_fit[0].data;
img=img.astype('d')
fid_fit.close()
dark=fits.getdata(dark_filename);
dark.astype('d')
img=img-dark
date_obs=datetime.datetime.strptime(date_obs_str,"%Y-%m-%dT%H:%M:%S")+datetime.timedelta(seconds=15)
print(date_obs)

2014-08-26 19:37:17


In [9]:
BS_coord=SkyCoord(CAT_6300BS[:,2], CAT_6300BS[:,3], frame='icrs', unit='deg');

altaz=BS_coord.transform_to(AltAz(obstime=date_obs, location=KEO_site,temperature=15*u.deg_C,pressure=1013*u.hPa,
                                       relative_humidity=0.5,obswl=630.0*u.nm))
BS_AzAlt=np.zeros((np.size(CAT_6300BS,0),2))
BS_AzAlt[:,0]=altaz.az.degree
BS_AzAlt[:,1]=altaz.alt.degree

In [10]:
BS_xy=np.zeros((np.size(CAT_6300BS,0),2))
BS_xy[:,0], BS_xy[:,1] = hor2pix_keo (BS_AzAlt[:,0]*np.pi/180,BS_AzAlt[:,1]*np.pi/180, az0_keo, alt0_keo, c_keo, d_keo)

In [11]:
# Catalog filtration
filt_mask=np.zeros(np.size(CAT_6300BS,0),dtype=bool)
for i in range(0,np.size(CAT_6300BS,0)):
    if BS_AzAlt[i,1]>=44.0 and BS_AzAlt[i,1]<=62.0:
#     if BS_AzAlt[i,1]>=62.0: #and BS_AzAlt[i,1]<=82.0:
        if BS_xy[i,0]>=1 and BS_xy[i,0]<=511:
            if BS_xy[i,1]>=1 and BS_xy[i,1]<=511:
                filt_mask[i]=True
BS_AzAlt_filt=BS_AzAlt[filt_mask,:]
BS_xy_filt=BS_xy[filt_mask,:]
CAT_6300BS_filt=CAT_6300BS[filt_mask,:]
CAT_SPTYPE_filt=CAT_SPTYPE[filt_mask]

In [12]:
np.size(BS_xy_filt,0)

92

In [13]:
CAT_6300BS_filt[:,0]

array([   1.,    5.,    6.,   11.,   12.,   13.,   14.,   15.,   17.,
         18.,   19.,   22.,   23.,   28.,   29.,   30.,   36.,   37.,
         40.,   47.,   48.,   58.,   89.,   90.,   97.,  132.,  259.,
        280.,  328.,  336.,  349.,  355.,  360.,  363.,  365.,  369.,
        370.,  377.,  379.,  385.,  387.,  388.,  390.,  392.,  393.,
        399.,  401.,  402.,  403.,  406.,  409.,  410.,  412.,  419.,
        420.,  424.,  425.,  426.,  431.,  438.,  442.,  443.,  447.,
        455.,  456.,  457.,  458.,  460.,  462.,  469.,  472.,  473.,
        476.,  478.,  486.,  487.,  497.,  498.,  507.,  508.,  514.,
        525.,  526.,  528.,  530.,  535.,  548.,  549.,  552.,  553.,
        564.,  565.])

In [14]:
fig, ax = plt.subplots()
# plt.imshow(img, cmap="gray", vmin=300, vmax=900)
plt.pcolormesh(img, cmap="gray", vmin=300, vmax=1000)
#plt.colorbar
plt.plot(BS_xy_filt[:,0],BS_xy_filt[:,1],color="r",marker=".", lw=0.,mec="r", mfc="none")
# ax.set_xlim([247+10,247-10]);
# ax.set_ylim([392+10,392-10]);

# ax.set_xlim([275,175]);
# ax.set_ylim([325,425]);

ax.set_xlim((511,1))
ax.set_ylim((511,1))

plt.show()

In [15]:
img_red_st_int=np.zeros(np.size(BS_xy_filt,0))
img_red_st_num=np.zeros(np.size(BS_xy_filt,0))
BS_adc_filt=np.zeros(np.size(BS_xy_filt,0))
area_rad=2
for j in range(np.size(BS_xy_filt,0)):    
    st_x=BS_xy_filt[j,0]
    st_y=BS_xy_filt[j,1]
    area=img[int(st_y)-area_rad:int(st_y)+area_rad+1, int(st_x)-area_rad:int(st_x)+area_rad+1]

    sum_temp=0.0
    num=0
    med=np.median(area)
    for i in range(len(area.flat)):
        if area.flat[i]>=1.3*med:
            num+=1
            sum_temp+=area.flat[i]-med
    img_red_st_int[j]=sum_temp
    img_red_st_num[j]=num
    if num>0:
        BS_adc_filt[j]=sum_temp/num

In [16]:
print(img_red_st_num)

[ 3.  1.  0.  1.  0.  0.  3.  0.  2.  0.  0.  1.  1.  0.  4.  0.  2.  3.
  0.  1.  1.  0.  0.  1.  0.  0.  0.  0.  4.  2.  0.  1.  4.  1.  1.  0.
  2.  0.  4.  2.  2.  2.  0.  0.  0.  6.  2.  2.  1.  1.  0.  0.  0.  2.
  3.  1.  1.  1.  2.  0.  4.  0.  1.  1.  2.  0.  1.  0.  1.  1.  0.  0.
  0.  0.  0.  2.  1.  2.  2.  0.  0.  0.  3.  2.  2.  3.  0.  0.  0.  0.
  0.  0.]


In [18]:
BS_relay=np.zeros(np.size(BS_xy_filt,0))
R_adc_coef=np.zeros(np.size(BS_xy_filt,0))
for i in range(np.size(BS_xy_filt,0)):
     if img_red_st_num[i]>3:
            BS_relay[i]=bright_relay_keo(CAT_6300BS_filt[i,5],img_red_st_num[i])
            if BS_adc_filt[i]>0 and BS_relay[i]>100:
                R_adc_coef[i]=BS_relay[i]/BS_adc_filt[i]

print(BS_relay)

In [19]:
fig,ax = plt.subplots()
plt.scatter(BS_relay,BS_adc_filt,s=np.pi*(img_red_st_num)**2)
ax.set_xlim((100, 1300))
ax.set_ylim((500, 5000))
plt.show()

In [20]:
print(R_adc_coef[R_adc_coef.nonzero()])
R_median=np.median(R_adc_coef[R_adc_coef.nonzero()])
print(R_median)

[ 0.45914529  0.36608309  0.30727433  0.36848217  0.32631271  0.35235592]
0.359219505386


In [21]:
fig,ax = plt.subplots()
plt.plot([0, 5000], [R_median, R_median], c='r', lw=2)
plt.scatter(BS_relay,R_adc_coef,s=np.pi*(img_red_st_num)**2)
plt.ylabel('Calibration coef. [R/ADCu]')
plt.xlabel('Star brightness per pixel [R]')
plt.title(date_obs, loc='left')
plt.title(R_median, loc='right')
ax.set_xlim((100, 1300))
ax.set_ylim((0, 0.5))
plt.show()
# plt.savefig(pic_save_name)